In [ ]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import time
import json


In [ ]:
url = "https://www.fotmob.com/api/leagueseasondeepstats?id=130&season=24524&type=players&stat=rating"
payload = {}
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36",
    "Referer": "https://www.fotmob.com/leagues/130/stats/season/24524/players/rating",
    "x-mas": "eyJib2R5Ijp7InVybCI6Ii9hcGkvbGVhZ3Vlc2Vhc29uZGVlcHN0YXRzP2lkPTEzMCZzZWFzb249MjQ1MjQmdHlwZT1wbGF5ZXJzJnN0YXQ9cmF0aW5nIiwiY29kZSI6MTc0NzQwODM3NDg0MywiZm9vIjoicHJvZHVjdGlvbjo4OTMzMDRhNDBhYmUyZTM4YzJmOTE1NTk4YzAzMDZmZjMzZjRmYjE1LXVuZGVmaW5lZCJ9LCJzaWduYXR1cmUiOiIxNUU0RjA2QjIyMEI2MDE1RjQ0RDI3REQ1M0Q4REQ3MyJ9",
    "sec-ch-ua": "\"Chromium\";v=\"136\", \"Google Chrome\";v=\"136\", \"Not.A/Brand\";v=\"99\"",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-ch-ua-mobile": "?0"
}

response = requests.request("GET", url, headers=headers, data=payload)
response.raise_for_status()
print("\nStatus code:", response.status_code)



Status code: 200


In [ ]:
rating=response.json()['statsData']

In [ ]:
ratings = pd.DataFrame(rating)
ratings.head()

,id,teamId,name,substatValue,statValue,rank,type
0,30981,960720,Lionel Messi,"{'value': 4, 'format': 'number', 'fractions': 0}","{'name': 'rating', 'value': 8.04, 'format': 'f...",1,players
1,358289,6580,Carles Gil,"{'value': 4, 'format': 'number', 'fractions': 0}","{'name': 'rating', 'value': 8.03, 'format': 'f...",2,players
2,589897,6603,Cristian Espinoza,"{'value': 3, 'format': 'number', 'fractions': 0}","{'name': 'rating', 'value': 8.02, 'format': 'f...",3,players
3,641344,1701119,Anders Dreyer,"{'value': 3, 'format': 'number', 'fractions': 0}","{'name': 'rating', 'value': 7.78, 'format': 'f...",4,players
4,862284,191716,Kai Wagner,"{'value': 1, 'format': 'number', 'fractions': 0}","{'name': 'rating', 'value': 7.73, 'format': 'f...",5,players


In [ ]:
ratings_test = []
for rat in rating:
  ratings_test.append({'id': rat['id'], 'team_id': rat['teamId'], 'name': rat['name'], 'rating': rat['statValue']['value']})

df=pd.DataFrame(ratings_test)
df


,id,team_id,name,rating
0,30981,960720,Lionel Messi,8.04
1,358289,6580,Carles Gil,8.03
2,589897,6603,Cristian Espinoza,8.02
3,641344,1701119,Anders Dreyer,7.78
4,862284,191716,Kai Wagner,7.73
...,...,...,...,...
466,621541,6637,John McCarthy,5.98
467,847753,6397,Tom Barlow,5.93
468,964391,6606,William Agada,5.93
469,627454,8259,Daniel Steres,5.88


In [ ]:
df.to_csv('ratsApril.csv', index=False)

In [ ]:
# MAKE PLAYER LINKS #

links = []
for index, row in ratings.iterrows():
  p_id = row['id']
  player_name = row['name'].lower().replace(' ', '-')

  link = f'https://www.fotmob.com/players/{p_id}/{player_name}'
  links.append(link)

In [ ]:
len(links)

471

In [ ]:
df['links'] = links

In [ ]:
df.head()

,id,team_id,name,rating,links
0,30981,960720,Lionel Messi,8.04,https://www.fotmob.com/players/30981/lionel-messi
1,358289,6580,Carles Gil,8.03,https://www.fotmob.com/players/358289/carles-gil
2,589897,6603,Cristian Espinoza,8.02,https://www.fotmob.com/players/589897/cristian...
3,641344,1701119,Anders Dreyer,7.78,https://www.fotmob.com/players/641344/anders-d...
4,862284,191716,Kai Wagner,7.73,https://www.fotmob.com/players/862284/kai-wagner


In [ ]:
def fetch_player_details(df):
    player_details = []

    for idx, row in df.iterrows():
        player_id = row['id']
        player_name = row['name']

        url = f"https://www.fotmob.com/api/playerData?id={player_id}"
        headers = {
            "sec-ch-ua-platform": "\"Windows\"",
            "Referer": row['links'],
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36",
            "sec-ch-ua": "\"Google Chrome\";v=\"135\", \"Not-A.Brand\";v=\"8\", \"Chromium\";v=\"135\"",
            "sec-ch-ua-mobile": "?0",
            "x-mas": "eyJib2R5Ijp7InVybCI6Ii9hcGkvcGxheWVyRGF0YT9pZD0zMDk4MSIsImNvZGUiOjE3NDc0MDkxNDM4MzQsImZvbyI6InByb2R1Y3Rpb246ODkzMzA0YTQwYWJlMmUzOGMyZjkxNTU5OGMwMzA2ZmYzM2Y0ZmIxNS11bmRlZmluZWQifSwic2lnbmF0dXJlIjoiQUZFQjg2Njc1RUUyODIwNDk1M0RDNzVENTY4RTlDNzkifQ=="
        }

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()

            position = None

            if 'positionDescription' in data and 'primaryPosition' in data['positionDescription']:
                position = data['positionDescription']['primaryPosition'].get('label', '')

            player_details.append({
                'id': player_id,
                'position': position
            })

            time.sleep(0.5)

        except Exception as e:
            print(f"Error processing {player_name}")

    details_df = pd.DataFrame(player_details)

    result_df = df.merge(details_df, on='id', how='left')

    return result_df

In [ ]:
testdrive = df.head(3)
testdrive

,id,team_id,name,rating,links
0,30981,960720,Lionel Messi,8.04,https://www.fotmob.com/players/30981/lionel-messi
1,358289,6580,Carles Gil,8.03,https://www.fotmob.com/players/358289/carles-gil
2,589897,6603,Cristian Espinoza,8.02,https://www.fotmob.com/players/589897/cristian...


In [ ]:
# Get position data #
pos = fetch_player_details(testdrive)

In [ ]:
pos

,id,team_id,name,rating,links,position,age
0,30981,960720,Lionel Messi,8.04,https://www.fotmob.com/players/30981/lionel-messi,Striker,None
1,358289,6580,Carles Gil,8.03,https://www.fotmob.com/players/358289/carles-gil,Attacking Midfielder,None
2,589897,6603,Cristian Espinoza,8.02,https://www.fotmob.com/players/589897/cristian...,Right Winger,None


In [ ]:
pos = fetch_player_details(df)

In [ ]:
pos.to_csv('mls_pos.csv', index=False)